In [2]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np

ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
# Initialize Mediapipe and PyAutoGUI
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
screen_width, screen_height = pyautogui.size()
camera = cv2.VideoCapture(0)

In [ ]:
# Smoothing parameters for Exponential Moving Average (EMA)
smooth_factor = 0.6  # Increased smoothing factor for smoother movement
ema_x, ema_y = 0, 0  # Initial EMA positions

In [ ]:
# Main loop
while True:
    success, image = camera.read()
    if not success:
        break

In [ ]:
# Flip the image for mirror effect and convert the color to RGB
    image = cv2.flip(image, 1)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_image)
    image_height, image_width, _ = image.shape

In [ ]:
    # Process hand landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


In [ ]:
# Get the index finger tip coordinates (landmark ID 8)
            index_finger_tip = hand_landmarks.landmark[8]
            x = int(index_finger_tip.x * image_width)
            y = int(index_finger_tip.y * image_height)

In [ ]:
 # Convert the coordinates to screen space
            screen_x = np.interp(x, (0, image_width), (0, screen_width))
            screen_y = np.interp(y, (0, image_height), (0, screen_height))

In [ ]:
 # Apply Exponential Moving Average (EMA) for smoothing
            ema_x = smooth_factor * screen_x + (1 - smooth_factor) * ema_x
            ema_y = smooth_factor * screen_y + (1 - smooth_factor) * ema_y

In [ ]:
 # Move the mouse to the smoothed EMA position
            pyautogui.moveTo(ema_x, ema_y)

In [ ]:
# Draw circle at index finger tip for visualization
            cv2.circle(image, (x, y), 10, (0, 255, 255), cv2.FILLED)


In [ ]:
 # Click detection using a threshold on vertical distance
            thumb_tip = hand_landmarks.landmark[4]
            thumb_x = int(thumb_tip.x * image_width)
            thumb_y = int(thumb_tip.y * image_height)

In [ ]:
 # Check distance between index finger tip and thumb tip
            distance = np.hypot(x - thumb_x, y - thumb_y)
            print(f"Distance: {distance}")


In [ ]:
 # If distance is less than threshold, perform a click
            if distance < 30:
                pyautogui.click()
                pyautogui.sleep(0.1)  # Add a small delay to prevent multiple clicks

In [ ]:
    # Display the resulting frame
    cv2.imshow("Hand Movement Video Capture", image)


In [ ]:
 # Exit the loop when 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

In [ ]:
# Release the camera and close all windows
camera.release()
cv2.destroyAllWindows()